In [2]:
import pandas as pd
from thunderpack import ThunderReader
from openpyxl import load_workbook


In [8]:
#get list of patients with missing death from excel file
#include patient ID and encounter ID in the list
df = pd.read_excel('/media/cdac-c-15/Thunderpack/Dropbox/cardiac arrest project/cardiac_filtered_orginal .xlsx')

missingDataPIDs = df[df['DateOfDeath'].isna()]['BDSPPatientID'].tolist()
missingDataEIDs = df[df['DateOfDeath'].isna()]['BDSPEncounterID'].tolist()
dfNoDeathDate = pd.DataFrame({
    'PatientID': missingDataPIDs,
    'EncounterID': missingDataEIDs
})

dfNoDeathDate.head()

,PatientID,EncounterID
0,111189188,538929570
1,111189188,538929570
2,111189188,538929570
3,111189188,538929570
4,111189188,538929570


In [9]:
contains_pair = ((dfNoDeathDate['PatientID'] == 111227152) & (dfNoDeathDate['EncounterID'] == 533711015)).any()
print(f"{contains_pair}")

#checking that the list contains the correct pairs. true for 2 different IDS
#bad ids check? passed for both lists
#good ids check? passed for both lists


False


In [10]:
# create list of patients that has no duplicates 
noDeathPID = dfNoDeathDate['PatientID']
noDeathPID = dfNoDeathDate.drop_duplicates()
print(len(noDeathPID))
noDeathPID.head()

911


,PatientID,EncounterID
0,111189188,538929570
12,111200203,597807882
13,111204957,561278739
18,111205248,740136037
19,111222562,574767840


In [11]:
# read into demographics file to make a dataframe that contains information about patients with no death date
reader = ThunderReader('/media/cdac-c-15/Thunderpack/Dropbox/zz_EHR_Thunderpacks/MGB/thunderpack_demographics_MGB')
dfs = []
for i in range(1, (len(list(reader.keys())))):
    df = reader[f'demographics_partition_{i}']
    dfx = df[df['BDSPPatientID'].isin(noDeathPID['PatientID'])]
    dfs.append(dfx)

In [12]:
filtered_icd_df = pd.concat(dfs, axis=0, ignore_index=True)     #puts data into rows and columns
print(len(df))      #print the number of notes found with those ICD codes
filtered_icd_df.head()      #print the first few lines of the filtered list of notes

1178760


,BDSPPatientID,DateOfBirth,DateOfDeath,DateOfDeathMARegistryData,PatientRace,EthnicGroupDSC,MaritalStatusDSC,ReligionDSC,LanguageDSC,VeteranStatusDSC,...,PrimaryCauseOfDeathDSC,UNOSPrimaryCauseOfDeathTXT,FirstContributoryCauseOfDeathDSC,UNOSContributoryCauseOfDeath01TXT,SecondContributoryCauseOfDeathDSC,UNOSContributoryCauseOfDeath02TXT,EducationLevelDSC,GenderIdentityDSC,SexAssignedAtBirthDSC,BDSPLastModifiedDTS
0,119106063,1963-06-28,None,None,Black or African American,Unavailable,Life Partner,Not Affiliated,English,"No, Never Served or Is Currently Active",...,None,None,None,None,None,NaN,Other,None,None,2022-07-17 17:57:14.1830000
1,114886961,2000-03-12,None,None,Other,Hispanic,Single,Not Affiliated,Spanish,"No, Never Served or Is Currently Active",...,None,None,None,None,None,NaN,Graduated - High School,Male,Male,2022-07-17 17:57:14.1830000
2,113203278,1971-07-09,None,None,Black or African American,Not Hispanic,Married/Civil Union,Muslim,English,"No, Never Served or Is Currently Active",...,None,None,None,None,None,NaN,Graduated - College,Male,Male,2022-07-17 17:57:14.1830000
3,115097977,1980-03-16,None,None,White,Not Hispanic,Single,Unavailable,English,Unavailable,...,None,None,None,None,None,NaN,Unavailable,Male,Male,2022-07-17 17:57:14.1830000
4,113470261,1991-01-27,None,None,White,Not Hispanic,Single,Roman Catholic,English,"No, Never Served or Is Currently Active",...,None,None,None,None,None,NaN,Graduated - High School,Female,Female,2022-07-17 17:57:14.1830000


In [15]:
#merge dfs to make dfs contain missing data
dfNoDeathDate = dfNoDeathDate.merge(filtered_icd_df[['BDSPPatientID', 'DateOfDeathMARegistryData']], left_on='PatientID', right_on='BDSPPatientID', how='left')
dfNoDeathDate.head()

,PatientID,EncounterID,BDSPPatientID,DateOfDeathMARegistryData
0,111189188,538929570,111189188,2018-06-16
1,111189188,538929570,111189188,2018-06-16
2,111189188,538929570,111189188,2018-06-16
3,111189188,538929570,111189188,2018-06-16
4,111189188,538929570,111189188,2018-06-16


In [16]:
contains_pair = ((dfNoDeathDate['PatientID'] == 112488669) & (dfNoDeathDate['EncounterID'] == 1561255390)).any()
print(f"{contains_pair}")

False


In [18]:
excel_df = pd.read_excel('/media/cdac-c-15/Thunderpack/Dropbox/cardiac arrest project/cardiac_filtered_orginal .xlsx', sheet_name='cardiac_filtered_new')

In [21]:
excel_df.head()
excel_df.to_csv('excel_csv.csv', index=False)

In [22]:
#merge excel sheet with existing data
# Assuming 'PatientID' and 'EncounterID' are columns in both DataFrames
# and we want to update the 'Data_Column' in the Excel DataFrame
merged_df = excel_df.merge(dfNoDeathDate, left_on=['BDSPPatientID', 'BDSPEncounterID'], right_on=['PatientID', 'EncounterID'], how='inner')
excel_df['DateOfDeath'] = merged_df['DateOfDeathMARegistryData'].combine_first(excel_df['DateOfDeath'])
merged_df.to_csv('merged_csv.csv', index=False)

excel_df.head()

,Unnamed: 0,group,BDSPPatientID,SiteID,BonoboIndex,sparcnetID,HashFolderName,HashFileName,BidsFolder,SessionID_new,...,bects,jme,jae,jeavons,sunflower,dravet,eses,cjd,angelman,jeavonssunflower
0,43477,9,111189188,S0001,NaN,pat0005,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,sub-S0001111189188,7,...,0,0,0,0,0,0,0,0,0,NaN
1,88576,9,111189188,S0001,NaN,pat0005,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,sub-S0001111189188,7,...,0,0,0,0,0,0,0,0,0,NaN
2,102055,9,111189188,S0001,NaN,pat0005,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,sub-S0001111189188,6,...,0,0,0,0,0,0,0,0,0,NaN
3,194219,9,111189188,S0001,NaN,pat0005,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,sub-S0001111189188,6,...,0,0,0,0,0,0,0,0,0,NaN
4,132878,9,111189188,S0001,NaN,pat0005,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,sub-S0001111189188,5,...,0,0,0,0,0,0,0,0,0,NaN


In [12]:
# Load the workbook and the specific sheet
wb = load_workbook('/media/cdac-c-15/Thunderpack/Dropbox/cardiac arrest project/cardiac_filtered_orginal.xlsx')
ws = wb['cardiac_filtered_new']

# Convert the updated DataFrame to a dictionary for easy updating
updated_data = excel_df.set_index(['BDSPPatientID', 'BDSPEncounterID'])['DateOfDeath'].to_dict()

# Update the existing column in the sheet
for row in ws.iter_rows(min_row=2, max_col=ws.max_column, max_row=ws.max_row):  # Adjust min_row if headers are elsewhere
    patient_id = row[2].value  # Assuming PatientID is in the first column
    encounter_id = row[25].value  # Assuming EncounterID is in the second column
    if (patient_id, encounter_id) in updated_data:
        row[19].value = updated_data[(patient_id, encounter_id)]  # Assuming ExistingColumn is in the third column


print(updated_data)

{(111189188, 538929570): '2018-06-16', (111200203, 597807882): '2018-06-16', (111204957, 561278739): '2018-06-16', (111205248, 740136037): '2018-06-16', (111222562, 574767840): '2018-06-16', (111227152, 533711015): '2018-06-16', (111244914, 539201634): '2018-06-16', (111245551, 740672463): '2018-06-16', (111249291, 1565487169): '2018-06-16', (111268824, 425933583): '2018-06-16', (111317668, 1602997059): '2018-06-16', (111317668, 1602996931): '2018-06-16', (111329437, 515817372): '2018-06-16', (111330137, 581416086): '2018-06-16', (111339883, 462355450): '2018-06-16', (111354808, 413190879): '2018-06-16', (111354808, 402825110): '2018-06-16', (111366484, 453016732): '2018-06-16', (111409348, 541577438): '2018-06-16', (111409469, 454043525): '2018-06-16', (111412824, 561061932): '2018-06-16', (111418364, 597799574): '2018-06-16', (111427501, 457522714): '2018-06-16', (111430326, 518956762): Timestamp('2019-03-16 00:00:00'), (111468571, 547866687): Timestamp('2019-08-04 00:00:00'), (11148

In [13]:
# Save the workbook
wb.save('/media/cdac-c-15/Thunderpack/Dropbox/cardiac arrest project/cardiac_filtered_orginal.xlsx')
print("Updated Excel DataFrame:")
print(updated_data)

Updated Excel DataFrame:
{(111189188, 538929570): '2018-06-16', (111200203, 597807882): '2018-06-16', (111204957, 561278739): '2018-06-16', (111205248, 740136037): '2018-06-16', (111222562, 574767840): '2018-06-16', (111227152, 533711015): '2018-06-16', (111244914, 539201634): '2018-06-16', (111245551, 740672463): '2018-06-16', (111249291, 1565487169): '2018-06-16', (111268824, 425933583): '2018-06-16', (111317668, 1602997059): '2018-06-16', (111317668, 1602996931): '2018-06-16', (111329437, 515817372): '2018-06-16', (111330137, 581416086): '2018-06-16', (111339883, 462355450): '2018-06-16', (111354808, 413190879): '2018-06-16', (111354808, 402825110): '2018-06-16', (111366484, 453016732): '2018-06-16', (111409348, 541577438): '2018-06-16', (111409469, 454043525): '2018-06-16', (111412824, 561061932): '2018-06-16', (111418364, 597799574): '2018-06-16', (111427501, 457522714): '2018-06-16', (111430326, 518956762): Timestamp('2019-03-16 00:00:00'), (111468571, 547866687): Timestamp('2019